# DataPath Example 4

This notebook covers somewhat more advanced examples for using `DataPath`s. It assumes that you understand 
the concepts presented in the previous example notebooks.

You should also read the ERMrest documentation and the derivapy wiki. There are more advanced concepts in this notebook that are demonstrated but not fully (re)explained here, as the concepts are explained in other documentation.

## Exampe Data Model
The examples require that you understand a little bit about the example catalog data model, which is based on the FaceBase project.

### Key tables
- `'dataset'` : represents a unit of data usually a `'study'` or `'experiment'`
- `'sample'` : a biosample
- `'assay'` : a bioassay (typically RNA-seq or ChIP-seq assays)

### Relationships
- `dataset <- sample`: A dataset may have one to many samples. I.e., there 
  is a foreign key reference from sample to dataset.
- `sample <- assay`: A sample may have one to many assays. I.e., there is a
  foreign key reference from assay to sample.

In [1]:
# Import deriva modules
from deriva.core import ErmrestCatalog, get_credential

In [2]:
# Connect with the deriva catalog
protocol = 'https'
hostname = 'www.facebase.org'
catalog_number = 1
credential = None
# If you need to authenticate, use Deriva Auth agent and get the credential
# credential = get_credential(hostname)
catalog = ErmrestCatalog(protocol, hostname, catalog_number, credential)

In [3]:
# Get the path builder interface for this catalog
pb = catalog.getPathBuilder()

# Get some local variable handles to tables for convenience
dataset = pb.isa.dataset
sample = pb.isa.sample
assay = pb.isa.assay

## Implicit DataPaths
**Proceed with caution**

For compactness, `Table` objects (and `TableAlias` objects) provide `DataPath`-like methods. E.g., `link(...)`, `filter(...)`, and `entities(...)`, which will implicitly create `DataPath`s rooted at the table and return the newly created path. These operations `return` the new `DataPath` rather than mutating the `Table` (or `TableAlias`) objects.

In [4]:
entities = dataset.filter(dataset.status > 2).entities()
len(entities)

71

### DataPath-like methods
The `DataPath`-like methods on `Table`s are essentially "wrapper" functions over the implicitly generated `DataPath` rooted at the `Table` instance. The wrappers include:
- `link(...)` generates a path, links the related table, and returns the generated path.
- `filter(...)` generate a path, adds a filter, and returns the generated path.
- `entities(...)` generate a path, (optionally) selects columns, and returns an entity set.

## Attribute Examples

### Example: selecting all columns of a table instance
Passing a table (or table instance) object to the `entities(...)` method will project all (i.e., `*`) of its attributes.

In [5]:
path = dataset.alias('D').path
path.link(sample).link(assay)
entities = path.entities()
print(len(entities))
print(path.uri)

171
https://www.facebase.org/ermrest/catalog/1/entity/D:=isa:dataset/sample:=isa:sample/assay:=isa:assay


In [6]:
entities = path.entities(path.D)
print(entities.uri)

https://www.facebase.org/ermrest/catalog/1/attribute/D:=isa:dataset/sample:=isa:sample/assay:=isa:assay/D:*


It is important to remember that the entities method returns a entity set based on the entity type of the last elmenent of the path. In this example that means the number of results will be determined by the number of unique rows in the assay table instance in the path created above, as the last link method used the assay table.  

In this example, we are only using attributes from dataset.  What this means is that even though the result of an entities method is always a entity set, since we are not including the attributes from assay in our result, it is possible that we may have duplicate rows, if the same assay is used in more then one dataset.

### Example: selecting from multiple table instances
More than one table instance may be selected in this manner and it can be mixed and matched with columns from other tables instances. 

In [7]:
entities = path.entities(path.D,
                         path.assay.fragmentation_method,
                         path.sample,
                         path.assay.isolation_protocol)
print (len(entities))
print(entities.uri)

171
https://www.facebase.org/ermrest/catalog/1/attribute/D:=isa:dataset/sample:=isa:sample/assay:=isa:assay/D:*,assay:fragmentation_method,sample:*,assay:isolation_protocol


 If you want to base the results on a different entity, you can introduce a table instance alias into the end of the path, before calling the entities function.  In this case, even though we are asking for the same attributes, we are getting the set of datasets, not the set of assays.  Also, since we are including the attributes from dataset in our query, we know that we will not be seeing any duplicate rows.

In [8]:
entities = path.D.entities(path.D,
                           path.assay.fragmentation_method,
                           path.sample,
                           path.assay.isolation_protocol)
print (len(entities))
print(entities.uri)

7
https://www.facebase.org/ermrest/catalog/1/attribute/D:=isa:dataset/sample:=isa:sample/assay:=isa:assay/$D/D:*,assay:fragmentation_method,sample:*,assay:isolation_protocol


## Filtering Examples

### Example: filter on `null` attribute
To test for a `null` attribute value, do an equality comparison against the `None` identity.

In [9]:
path = dataset.link(sample).filter(sample.gene == None)
print(path.uri)
print(len(path.entities()))

https://www.facebase.org/ermrest/catalog/1/entity/dataset:=isa:dataset/sample:=isa:sample/gene::null::
1216


### Example: advanced text filters
Deriva supports advanced text filters for regular expressions (`regexp`), case-instansitive regexp (`ciregexp`), and text search (`ts`). You may have to review your text and full-text indexes in your ERMrest catalog before using these features.

In [10]:
path = dataset.filter(dataset.description.ciregexp('palate'))
print(path.uri)
print(len(path.entities()))

https://www.facebase.org/ermrest/catalog/1/entity/dataset:=isa:dataset/description::ciregexp::palate
26


### Example: negate a filter
Use the "inverse" ('`~`') operator to negate a filter. Negation works against simple comparison filters as demonstrated above as well as on logical operators to be discussed next. You must wrap the comparison or logical operators in an extra parens to use the negate operation, e.g., "`~ (...)`". 

In [11]:
path = dataset.filter( ~ (dataset.description.ciregexp('palate')) )
print(path.uri)
print(len(path.entities()))

https://www.facebase.org/ermrest/catalog/1/entity/dataset:=isa:dataset/!(description::ciregexp::palate)
693


### Example: filters with logical operators
This example shows how to combine two comparisons with a conjuncting (i.e., `and` operator). Because Python's logical-and (`and`) keyword cannot be overloaded, we instead overload the bitwise-and (`&`) operator. This approach has become customary among many similar data access libraries.

In [12]:
path = dataset.link(sample).link(assay).filter(
    ((assay.markers == 'histology') & (assay.selection == 'H3K27AC')))

print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/dataset:=isa:dataset/sample:=isa:sample/assay:=isa:assay/(markers=histology)&(selection=H3K27AC)


In [13]:
path.entities().dataframe

,RCB,RCT,RID,RMB,RMT,alignment_id,dataset,fragmentation_method,id,isolation_protocol,...,reagent_source,replicate,sample,sample_composition,sample_purification,sample_type,selection,strandednes,target_of_assay,tracks_id
0,None,2017-09-22T17:33:18.797126-07:00,26531,None,2017-10-18T19:59:03.14624-07:00,59,14068,Sonication,19,,...,Active Motif,1,2,maxillary process,excision,2,H3K27AC,None,None,39
1,None,2017-09-22T17:33:18.797126-07:00,26535,None,2017-10-18T19:59:03.14624-07:00,45,14068,Sonication,5,,...,Active Motif,1,1,medial nasal process,excision,2,H3K27AC,None,None,25
2,None,2017-09-22T17:33:18.797126-07:00,26541,None,2017-10-18T19:59:03.14624-07:00,52,14068,Sonication,12,,...,Active Motif,1,4,mandibular process,excision,2,H3K27AC,None,None,32
3,None,2017-09-22T17:33:18.797126-07:00,26544,None,2017-10-18T19:59:03.14624-07:00,54,14068,Sonication,14,,...,Active Motif,1,3,latero nasal process,excision,2,H3K27AC,None,None,34


### Example: combine conjunction and disjunctions in filters
Similar to the prior example, the filters allow combining of conjunctive and disjunctive operators. Like the bitwise-and operator, we also overload the bitwise-or (` | `) operator because the logical-or (`or`) operatar cannot be overloaded.

In [14]:
path = dataset.link(sample).link(assay).filter(
    ((assay.markers == 'histology') & (assay.selection == 'H3K27AC')) |
    ((assay.sample_type == 3) & (assay.selection == 'genomicDNA')))

print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/dataset:=isa:dataset/sample:=isa:sample/assay:=isa:assay/((markers=histology)&(selection=H3K27AC));((sample_type=3)&(selection=genomicDNA))


In [15]:
path.entities().dataframe

,RCB,RCT,RID,RMB,RMT,alignment_id,dataset,fragmentation_method,id,isolation_protocol,...,reagent_source,replicate,sample,sample_composition,sample_purification,sample_type,selection,strandednes,target_of_assay,tracks_id
0,None,2017-09-22T17:33:18.797126-07:00,26531,None,2017-10-18T19:59:03.14624-07:00,59,14068,Sonication,19,,...,Active Motif,1,2,maxillary process,excision,2,H3K27AC,None,None,39.0
1,None,2017-09-22T17:33:18.797126-07:00,26533,None,2017-10-18T19:59:03.14624-07:00,47,14068,Sonication,7,,...,NA,4,3,latero nasal process,excision,3,genomicDNA,None,None,27.0
2,None,2017-09-22T17:33:18.797126-07:00,26535,None,2017-10-18T19:59:03.14624-07:00,45,14068,Sonication,5,,...,Active Motif,1,1,medial nasal process,excision,2,H3K27AC,None,None,25.0
3,None,2017-09-22T17:33:18.797126-07:00,26538,None,2017-10-18T19:59:03.14624-07:00,49,14068,Sonication,9,,...,NA,4,4,mandibular process,excision,3,genomicDNA,None,None,29.0
4,None,2017-09-22T17:33:18.797126-07:00,26541,None,2017-10-18T19:59:03.14624-07:00,52,14068,Sonication,12,,...,Active Motif,1,4,mandibular process,excision,2,H3K27AC,None,None,32.0
5,None,2017-09-22T17:33:18.797126-07:00,26544,None,2017-10-18T19:59:03.14624-07:00,54,14068,Sonication,14,,...,Active Motif,1,3,latero nasal process,excision,2,H3K27AC,None,None,34.0
6,None,2017-09-22T17:33:18.797126-07:00,26546,None,2017-10-18T19:59:03.14624-07:00,42,14068,Sonication,2,,...,NA,4,1,medial nasal process,excision,3,genomicDNA,None,None,22.0
7,None,2017-09-22T17:33:18.797126-07:00,26549,None,2017-10-18T19:59:03.14624-07:00,56,14068,Sonication,16,,...,NA,4,2,maxillary process,excision,3,genomicDNA,None,None,36.0
8,None,2017-09-22T17:33:18.797126-07:00,26554,None,2017-10-18T19:59:03.14624-07:00,61,14130,Sonication,24,None,...,NA,4,1088,face,Excision,3,genomicDNA,None,None,NaN
9,None,2017-09-22T17:33:18.797126-07:00,26559,None,2017-10-18T19:59:03.14624-07:00,63,14130,Sonication,29,None,...,NA,4,1089,face,Excision,3,genomicDNA,None,None,NaN


### Example: filtering at different stages of the path
Filtering a path does not have to be done at the end of a path. In fact, the initial intention of the ERMrest URI was to mimick "RESTful" semantics where a RESTful "resource" is identified, then filtered, then a "sub-resource" is identified, and then filtered, and so on.

In [16]:
path = dataset.filter(dataset.release_date >= '2017-01-01') \
    .link(sample).filter(sample.species == 2) \
    .link(assay).filter(assay.selection == 'totalRNA')
    
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/dataset:=isa:dataset/release_date::geq::2017-01-01/sample:=isa:sample/species=2/assay:=isa:assay/selection=totalRNA


In [17]:
path.entities().dataframe

,RCB,RCT,RID,RMB,RMT,alignment_id,dataset,fragmentation_method,id,isolation_protocol,...,reagent_source,replicate,sample,sample_composition,sample_purification,sample_type,selection,strandednes,target_of_assay,tracks_id
0,None,2017-09-22T17:33:18.797126-07:00,26532,None,2017-10-18T19:59:03.14624-07:00,60,14068,Fragmentation Buffer from Illumina,20,,...,Illumina,5,4,mandibular process,excision,1,totalRNA,None,None,40
1,None,2017-09-22T17:33:18.797126-07:00,26536,None,2017-10-18T19:59:03.14624-07:00,46,14068,Fragmentation Buffer from Illumina,6,,...,Illumina,5,3,latero nasal process,excision,1,totalRNA,None,None,26
2,None,2017-09-22T17:33:18.797126-07:00,26545,None,2017-10-18T19:59:03.14624-07:00,41,14068,Fragmentation Buffer from Illumina,1,,...,Illumina,5,1,medial nasal process,excision,1,totalRNA,None,None,21
3,None,2017-09-22T17:33:18.797126-07:00,26548,None,2017-10-18T19:59:03.14624-07:00,55,14068,Fragmentation Buffer from Illumina,15,,...,Illumina,5,2,maxillary process,excision,1,totalRNA,None,None,35


## Linking Examples

### Example: explicit column links
Up until now, the examples have shown how to link entities via _implicit_ join predicates. That is, we knew there existed a foriegn key reference constraint between foreign keys of one entity and keys of another entity. We needed only to ask ERMrest to link the entities in order to get the linked set.

The problem with implicit links is that it become _ambiguous_ if there are more than one foreign key reference between tables. To support these situations, ERMrest and the `DataPath`'s `link(...)` method can specify the columns to use for the link condition, explicitly.

The structure of the `on` clause is:
- an equality comparison operation where
- the _left_ operand is a column of the _left_ table instance which is also the path _context_ before the link method is called, and
- the _right_ operand is a column of the _right_ table instance which is the table _to be linked_ to the path.

In [18]:
path = dataset.link(sample, on=(dataset.id==sample.dataset))
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/dataset:=isa:dataset/sample:=(id)=(isa:sample:dataset)


**IMPORTANT** Not all tables are related by foreign key references. ERMrest does not allow arbitrary relational joins. Tables must be related by a foreign key reference in order to link them in a data path.

In [19]:
path.entities().fetch(limit=3).dataframe

,RCB,RCT,RID,RMB,RMT,_keywords,anatomy,collection_date,dataset,gender,...,mutation,origin,phenotype,replicate,replicate_group,species,specimen,stage,strain,theiler_stage
0,None,2017-09-22T17:33:18.797126-07:00,14312,None,2017-09-22T17:33:18.797126-07:00,None,173,2014-08-25,14139,None,...,None,None,None,None,None,2,None,5,3,25
1,None,2017-09-22T17:33:18.797126-07:00,14313,None,2017-09-22T17:33:18.797126-07:00,None,133,2014-08-25,14139,None,...,None,None,None,None,None,2,None,5,3,25
2,None,2017-09-22T17:33:18.797126-07:00,14314,None,2017-09-22T17:33:18.797126-07:00,None,139,2014-08-25,14139,None,...,None,None,None,None,None,2,None,5,3,25


### Example: explicit column links combined with table aliasing
As usual, table instances are generated automatically unless we provide a table alias.

In [20]:
path = dataset.link(sample.alias('S'), on=(dataset.id==sample.dataset))
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/dataset:=isa:dataset/S:=(id)=(isa:sample:dataset)


Notice that we cannot use the alias right away in the `on` clause because it was not _bound_ to the path until _after_ the `link(...)` operation was performed.

### Example: links with "outer join" semantics
Up until now, the examples have shown "`link`s" with _inner join_ semantics. _Outer join_ semantics can be expressed as part of explicit column links, and _only_ when using explicit column links.

The `link(...)` method accepts a "`join_type`" parameter, i.e., "`.link(... join_type=TYPE)`", where _TYPE_ may be `'left'`, `'right'`, `'full'`, and defaults to `''` which indicates inner join type.

By '`right`' outer joining in the link from `'sample'` to `'dataset`', the following path gives us a reference to `'dataset'` entities that _may or may not_ have any samples with assays.

In [21]:
path = assay.link(sample.alias('S'), on=assay.sample==sample.id) \
            .link(dataset.alias('D'), sample.dataset==dataset.id, join_type='right')

# path instances were not bound until the above line was executed

entities = path.entities(path.D.accession, path.D.title, path.S.species, path.S.stage)

print(entities.uri)
len(entities)

https://www.facebase.org/ermrest/catalog/1/attribute/assay:=isa:assay/S:=(sample)=(isa:sample:id)/D:=right(dataset)=(isa:dataset:id)/D:accession,D:title,S:species,S:stage


719

We can see above that we have a full set of datasets _whether or not_ they have any samples with assays. For further evidence, we can convert to a DataFrame and look at a slice of its entries. Note that the sample's 'species' and 'stage' attributes do not exist for some entities (i.e., `NaN`) because those entities did not exist for the join condition.

In [22]:
entities.dataframe[700:]

,accession,species,stage,title
700,FB00000884,NaN,NaN,Skull vault formation in individual WT AB Fish
701,FB00000885,NaN,NaN,Skull vault formation in individual WT AB Fish
702,FB00000886,NaN,NaN,Skull vault formation in individual WT AB Fish
703,FB00000887,NaN,NaN,Skull vault formation in individual WT AB Fish
704,FB00000888,NaN,NaN,Skull vault formation in individual WT AB Fish
705,FB00000889,NaN,NaN,Skull vault formation in individual WT AB Fish
706,FB00000890,NaN,NaN,Skull vault formation in individual WT AB Fish
707,FB00000891,2.0,5.0,Temporal analysis of ectoderm and mesenchyme e...
708,FB00000892,NaN,NaN,3D White Light Photogrammetry Images of North ...
709,FB00000902,2.0,25.0,Transcriptome Atlases of the Craniofacial Sutures


## Faceting Examples
You may have noticed that in the examples above, the 'species' and 'stage' attributes are numeric. These are internal "primary keys" that are not meaningful and difficult to filter on. We may want to construct filters on our datasets based on these categories. This can be used for "faceted search" modes and can be useful even within the context of programmatic access to data in the catalog.

### Example: faceting on "related" tables
Let's say we want to find all of the samples in our catalog where their species are 'Mus musculus' and their age stage are 'E10.5'.

We need to extend our understanding of the data model with the following tables that are related to '`sample`'.
- `isa.sample.species -> vocabulary.species.id`: the sample table has a foreign key reference to the '`species`' table.
- `isa.sample.stage -> vocabulary.stage.id`: the sample table has a foreign key reference to the '`stage`' table.

We may say that `species` and `stage` are _related_ to the `sample` table in the sense that `sample` has a direct foreign key relationship from it to them.

For convenience, we will get local variables for the species and stage tables.

In [23]:
species = pb.vocabulary.species
stage = pb.vocabulary.stage

First, let's link samples with species and filter on the term "Mus musculus" (i.e., "mouse").

In [24]:
path = sample.alias('S').link(species).filter(species.term == 'Mus musculus')
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/S:=isa:sample/species:=vocabulary:species/term=Mus%20musculus


Now the _context_ of the path is the `species` table instance, but we need to link from the `sample` to the age `stage` table.

To do so, we reference the `sample` table instance, in this case using its alias `S`. Then we link off of that table instance which updates the `path` itself.

In [25]:
path.S.link(stage).filter(stage.term == 'E10.5')
print(path.uri)

https://www.facebase.org/ermrest/catalog/1/entity/S:=isa:sample/species:=vocabulary:species/term=Mus%20musculus/$S/stage:=vocabulary:stage/term=E10.5


Now, the path _context_ is the age `stage` table instance, but we wanted to get the entities for the `sample` table. To do so, again we will reference the `sample` table instance by the alias `S` we used. From there, we will call the `entities(...)` method to get the samples.

In [26]:
entities = path.S.entities(path.S.id,
                           path.S.collection_date,
                           species=path.species.term,
                           species_iri=path.species.iri,
                           stage=path.stage.term,
                           stage_iei=path.stage.iri)
print(entities.uri)

https://www.facebase.org/ermrest/catalog/1/attribute/S:=isa:sample/species:=vocabulary:species/term=Mus%20musculus/$S/stage:=vocabulary:stage/term=E10.5/$S/S:id,S:collection_date,species:=species:term,species_iri:=species:iri,stage:=stage:term,stage_iei:=stage:iri


In [27]:
entities.dataframe

,collection_date,id,species,species_iri,stage,stage_iei
0,2015-07-14,1331,Mus musculus,http://purl.bioontology.org/ontology/NCBITAXON...,E10.5,http://purl.org/sig/ont/cmo/cmo0008759
1,2015-07-14,1332,Mus musculus,http://purl.bioontology.org/ontology/NCBITAXON...,E10.5,http://purl.org/sig/ont/cmo/cmo0008759
2,2015-07-14,1333,Mus musculus,http://purl.bioontology.org/ontology/NCBITAXON...,E10.5,http://purl.org/sig/ont/cmo/cmo0008759
3,2015-07-14,1334,Mus musculus,http://purl.bioontology.org/ontology/NCBITAXON...,E10.5,http://purl.org/sig/ont/cmo/cmo0008759
4,2015-07-14,1335,Mus musculus,http://purl.bioontology.org/ontology/NCBITAXON...,E10.5,http://purl.org/sig/ont/cmo/cmo0008759
5,2015-07-14,1336,Mus musculus,http://purl.bioontology.org/ontology/NCBITAXON...,E10.5,http://purl.org/sig/ont/cmo/cmo0008759
